In [1]:
# pip install -i https://pypi.tuna.tsinghua.edu.cn/simple agentscope --pre --user
# https://www.bilibili.com/video/BV14z421Q7DN/?spm_id_from=333.880.my_history.page.click&vd_source=fac9279bd4e33309b405d472b24286a8

In [50]:
import os
import sys
from typing import Any
import json
import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import torch as th
import agentscope
from agentscope import msghub
from agentscope.agents import (UserAgent, DialogAgent)
from agentscope.message import Msg
from agentscope.service import (ServiceToolkit, ServiceResponse)

In [2]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(th.__version__)
print(th.version.cuda)
print(agentscope.__version__)

device = cuda; devive_cnt = 1
2.2.0+cu118
11.8
0.0.6alpha2


## step-1: 准备模型配置文件

In [9]:
lst_model_config = [
    {
        "config_name": "ollama_chat-qwen2:7b",
        "model_type": "ollama_chat",
        "model_name": "qwen2:7b",
        "options": {"temperature": 0.7},
        "keep_alive": "5m",
        # "api_key": "xxx",
        # "organization": "xxx",
    }
]  # 可配置多个模型

In [10]:
agentscope.init(model_configs=lst_model_config)
# agentscope.init(model_configs="./model_config.json")

2024-08-18 13:49:45.294 | WARNING  | agentscope.models:read_model_configs:179 - config_name [ollama_chat-qwen2:7b] already exists.
2024-08-18 13:49:45.295 | INFO     | agentscope.models:read_model_configs:186 - Load configs for model wrapper: ollama_chat-qwen2:7b


[]

## step-2: 连接 AgentScope

In [11]:
agent = DialogAgent(
    name="assistant",
    sys_prompt="You are a helpful assistant.",
    model_config_name="ollama_chat-qwen2:7b"
)

2024-08-18 13:52:32.909 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [ollama_chat-qwen2:7b]


In [48]:
request = Msg(name="Lukas", content="你是谁？", role="user")
# request = Msg(name="Bob", content="What about this picture I took?", url="/path/to/picture.jpg")
response = agent(request)

assistant: 我是一个人工智能助手，专门用来提供信息、解答问题和进行日常对话。如果您对制定出行计划还有其他疑问或者需要进一步的建议，请随时告诉我。祝您旅途愉快！


In [36]:
print(json.dumps(response, indent=4))

{
    "id": "cfc1de10d1f942ed8788827ddb7f0470",
    "timestamp": "2024-08-18 14:00:07",
    "name": "assistant",
    "content": "\u6211\u662f\u60a8\u7684\u667a\u80fd\u52a9\u624b\uff0c\u4e00\u4e2a\u63d0\u4f9b\u5e2e\u52a9\u3001\u89e3\u7b54\u95ee\u9898\u4ee5\u53ca\u4e0e\u60a8\u8fdb\u884c\u7b80\u5355\u5bf9\u8bdd\u7684\u4eba\u5de5\u667a\u80fd\u7cfb\u7edf\u3002\u8bf7\u95ee\u60a8\u9700\u8981\u4ec0\u4e48\u5e2e\u52a9\u5462\uff1f",
    "role": "assistant",
    "url": null,
    "metadata": null
}


## step-3: 工具调用（未完）

In [43]:
def bing_search(
    query: str, 
    api_key: str, 
    num_results: int = 10, 
    **kwargs: Any
    ) -> ServiceResponse:
    """Search question in Bing Search API and return the searching results.

    Args:
        query (str): The search query string.
        api_key (str): The API key provided for authenticating with the Bing Search API.
        num_results (int, optional): The number of searching results. Defaults to 10.

    Returns:
        ServiceResponse: _description_
    """
    print(f"Calling web_search function with query {query}")
    

In [44]:
service_toolkit = ServiceToolkit()

In [45]:
service_toolkit.add(
    bing_search,
    api_key="xxx",
    num_results=10
)

## step-4: Agent 间通信

In [49]:
agent_lukas = DialogAgent(
    name="Lukas",
    sys_prompt="You are husband.",
    model_config_name="ollama_chat-qwen2:7b"
)

agent_s33 = DialogAgent(
    name="S33",
    sys_prompt="You are wife.",
    model_config_name="ollama_chat-qwen2:7b"
)

2024-08-18 14:37:40.854 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [ollama_chat-qwen2:7b]
2024-08-18 14:37:40.859 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [ollama_chat-qwen2:7b]


In [51]:
msg = Msg(name="son", content="Dad, Mum, I'm back.")

with msghub(participants=[agent_lukas, agent_s33], announcement=msg) as hub:
    agent_lukas()
    agent_s33()

2024-08-18 14:42:22.442 | WARNING  | agentscope.message:__init__:150 - A new field `role` is newly added to the message. Please specify the role of the message. Currently we use a default "assistant" value.
Lukas: Welcome home, son! How was your day? Did anything interesting happen while you were out?
S33: Hello, sweetheart. It's nice to see you again. Your dad and I were just talking about how your day went. Is there something specific you want to share or should we wait until you're ready to talk about it?


In [53]:
print(json.dumps(agent_lukas.memory.get_memory(), indent=4))

[
    {
        "id": "267cb91344954d84b04cd76dfb290edd",
        "timestamp": "2024-08-18 14:42:22",
        "name": "son",
        "content": "Dad, Mum, I'm back.",
        "role": "assistant",
        "url": null,
        "metadata": null
    },
    {
        "id": "1f3bdffea40a45dd859ad037eda00ccf",
        "timestamp": "2024-08-18 14:42:25",
        "name": "Lukas",
        "content": "Welcome home, son! How was your day? Did anything interesting happen while you were out?",
        "role": "assistant",
        "url": null,
        "metadata": null
    },
    {
        "id": "41dc14178dd445e5a8a8ada825d0e7c6",
        "timestamp": "2024-08-18 14:42:26",
        "name": "S33",
        "content": "Hello, sweetheart. It's nice to see you again. Your dad and I were just talking about how your day went. Is there something specific you want to share or should we wait until you're ready to talk about it?",
        "role": "assistant",
        "url": null,
        "metadata": null
    }
]

In [ ]:
with msghub(participants=[agent_lukas, agent_s33], announcement=request) as hub:
    agent_lukas()
    agent_s33()
    
    hub.add(agent_3)
    hub.delete(agent_s33)
    hub.broadcast(msg=Msg(name="son", content="Dad, Mum, I'm leaving."))
    
    agent_3()